In [16]:
import os

homeDIR = "C:/projects/AvitoML/data/"

os.environ["MKL_THREADING_LAYER"] = "GNU"
import numpy as np
import random
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.metrics import accuracy_score
from nltk.stem import SnowballStemmer
import pymorphy2
from gensim.models.keyedvectors import KeyedVectors
import json
import re
#from n05_train_xgb import preproc

morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer('russian')
skip_words = []

SEED = 1369
random.seed(1369)
np.random.seed(1369)

MAX_SEQUENCE_LENGTH = 384
MAX_NB_WORDS = 7040
EMBEDDING_DIM = 300
NUM_CLASS = 30

SAMPLE_SIZE = 1000

In [11]:
df = pd.read_csv(homeDIR+'train.csv')
print(df.head())
print(df.shape) ##489517

   item_id                                              title  \
0        0                                            Картина   
1        1                        Стулья из прессованной кожи   
2        2                                 Домашняя мини баня   
3        3  Эксклюзивная коллекция книг "Трансаэро" + подарок   
4        4                                       Ноутбук aser   

                                         description    price  category_id  
0                         Гобелен. Размеры 139х84см.   1000.0           19  
1  Продам недорого 4 стула из светлой прессованно...   1250.0           22  
2  Мини баня МБ-1(мини сауна), предназначена для ...  13000.0           37  
3  Продам эксклюзивную коллекцию книг, выпущенную...   4000.0           43  
4  Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...  19000.0            1  
(489517, 5)


In [12]:
data = pd.DataFrame()
data['title'] = df['title']
data['description'] = df['description']
data['price'] = df['price']
data['item_id'] = df['item_id']
data['category_id'] = df['category_id']
example = data['title'][0]

In [10]:
def preproc3(text):
    tokens = []
    text = re.sub(u"[^a-zа-я0-9]", " ", text.lower())
    text = re.sub(u"\s+", " ", text)
    for token in re.split(' ', text.lower()):
        if len(token) > 1:
            norm_token = stemmer.stem(token)
            if norm_token in skip_words:
                continue
            tokens.append(norm_token)
    if len(tokens) == 0:
        return 'None'
    return ' '.join(tokens)

print(preproc3(example))

стоик журнальн стал


In [23]:
x = np.array([preproc3(el) for el in data['title'][:SAMPLE_SIZE].tolist()])
y = np.array(data['category_id'][:SAMPLE_SIZE])
print(x.shape, y.shape)
print(MAX_SEQUENCE_LENGTH)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens')

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels_categ = to_categorical(y)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels_categ.shape)

    


(1000,) (1000,)
384
Found 2145 unique tokens
Shape of data tensor: (1000, 384)
Shape of label tensor: (1000, 54)


In [24]:
print( data)

[[   0    0    0 ...,    0    0   68]
 [   0    0    0 ...,    7  469   53]
 [   0    0    0 ...,   93   69   94]
 ..., 
 [   0    0    0 ...,   23 2144  468]
 [   0    0    0 ...,   50  167  441]
 [   0    0    0 ...,  357 2145  118]]


In [26]:
word2vec_model = KeyedVectors.load_word2vec_format(homeDIR+'ruscorpora_upos_skipgram_300_5_2018.vec', binary=False)


In [27]:
embeddings_index = {}
for word in word2vec_model.vocab:
    embeddings_index[word] = word2vec_model[word]
print('Found %s word vectors.' % len(embeddings_index))


Found 195071 word vectors.


In [28]:
 word2vec_model.vocab

{'мочь_VERB': <gensim.models.keyedvectors.Vocab at 0x17e4eebfb00>,
 'год_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4eebf400>,
 'xx_NUM': <gensim.models.keyedvectors.Vocab at 0x17e4eebf4e0>,
 'человек_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4eebf518>,
 'сказать_VERB': <gensim.models.keyedvectors.Vocab at 0x17e4eebf0b8>,
 'говорить_VERB': <gensim.models.keyedvectors.Vocab at 0x17e4eebf2b0>,
 'время_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4eebf048>,
 'xxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x17e4eebf3c8>,
 'знать_VERB': <gensim.models.keyedvectors.Vocab at 0x17e4eebfcc0>,
 'самый_ADJ': <gensim.models.keyedvectors.Vocab at 0x17e7adc7320>,
 'дело_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4f9f2898>,
 'делать_VERB': <gensim.models.keyedvectors.Vocab at 0x17e4f9f2710>,
 'день_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4f9f2828>,
 'жизнь_NOUN': <gensim.models.keyedvectors.Vocab at 0x17e4f9f22e8>,
 'видеть_VERB': <gensim.models.keyedvectors.Vocab at